In [1]:
(ns trachenberg.notebook.playground
    (:require [taoensso.encore :as e]
              [clj-time.core :as dt]
              [clj-time.coerce :as dtc]
              [clj-time.format :as dtf]
              [net.cgrand.xforms :as x]
              [kixi.stats.core :as kixi]
              [thi.ng.geom.viz.core :as viz]
              [thi.ng.geom.svg.core :as svg]
              [thi.ng.color.core :as col]
              [incanter.core :as i]
              [incanter.charts :as ic]
              [clojupyter.misc.display :as display]
              [trachenberg.trump :as trump]
              [trachenberg.wikidata :as wikidata]
              [trachenberg.language :as lang]
              [trachenberg.stock :as stock]))
              

In [2]:
(defn export-viz
  [spec]
  (->> spec
       (viz/svg-plot2d-cartesian)
       (svg/svg {:width 900 :height 600})
       (svg/serialize)
       (display/make-html)))

#'trachenberg.notebook.playground/export-viz

In [9]:
(def trades (stock/get-trades-with-change {:candles 30}))
(count trades)

55

In [4]:
(display/hiccup-html
    [:div (for [m trades]
              [:div
               [:div (:text m)]
               [:ul 
                [:li 
                 [:span (dtc/to-string (:created-at m))]
                 [:ul
                  [:li "company " [:b (:company m)]]
                  [:li "ticker " [:b (:ticker m)]]
                  [:li "sentiment " [:b (:sentiment m)]]
                  [:li "change " [:b (e/round2 (:percent-change m))]]
                  [:li "profit " [:b (e/round2 (:profit m))]]]]] 
               [:br]])])

Signing orders to move forward with the construction of the Keystone XL and Dakota Access pipelines in the Oval Off… https://t.co/aOxmfO0vOK 2017-01-24T17:49:17.000Z company Dakota Access Pipeline ticker ETP sentiment -0.2 change 0.01 profit -0.01 Great meeting with Ford CEO Mark Fields and General Motors CEO Mary Barra at the @WhiteHouse today. https://t.co/T0eIgO6LP8 2017-01-25T00:46:57.000Z company Ford ticker F sentiment 0.4 change -0.01 profit -0.01 Great meeting with Ford CEO Mark Fields and General Motors CEO Mary Barra at the @WhiteHouse today. https://t.co/T0eIgO6LP8 2017-01-25T00:46:57.000Z company Detroit Diesel ticker BLK sentiment 0.4 change 0.01 profit 0.01 Thank you Brian Krzanich, CEO of @Intel. A great investment ($7 BILLION) in American INNOVATION and JOBS!… https://t.co/oicfDsPKHQ 2017-02-08T19:22:33.000Z company Intel ticker INTC sentiment 0.3 change -0.03 profit -0.03 Obamacare continues to fail. Humana to pull out in 2018. Will repeal, replace & save healthcare for ALL Americans. 
https://t.co/glWEQ0lNR4 2017-02-14T22:50:33.000Z company Humana ticker HUM sentiment -0.1 change 0.0 profit 0.0 Aetna CEO: Obamacare in 'Death Spiral' #RepealAndReplace 
https://t.co/dmHL7xIEQv 2017-02-15T21:34:05.000Z company Aetna ticker AET sentiment -0.2 change -0.01 profit 0.01 Great optimism for future of U.S. business, AND JOBS, with the DOW having an 11th straight record close. Big tax & regulation cuts coming! 2017-02-25T13:27:04.000Z company Dow Jones Industrial Average ticker NWSA sentiment 0.3 change 0.02 profit 0.02 Wow, @FoxNews just reporting big news. Source: "Official behind unmasking is high up. Known Intel official is responsible. Some unmasked.... 2017-04-01T16:50:36.000Z company Intel ticker INTC sentiment 0.1 change 0.0 profit 0.0 Death spiral!
'Aetna will exit Obamacare markets in VA in 2018, citing expected losses on INDV plans this year'
https://t.co/5YnzDitF8r 2017-05-04T12:28:39.000Z company Aetna ticker AET sentiment -0.3 change 0.07 profit -0.07 Thoughts and prayers with the sailors of USS Fitzgerald and their families. Thank you to our Japanese allies for th… https://t.co/BBvmTFxVkk 2017-06-17T14:08:48.000Z company Bath Iron Works ticker GD sentiment 0.3 change 0.06 profit 0.06 The #AmazonWashingtonPost, sometimes referred to as the guardian of Amazon not paying internet taxes (which they should) is FAKE NEWS! 2017-06-28T13:06:14.000Z company Amazon ticker AMZN sentiment -0.7 change -0.02 profit 0.02 Crazy Joe Scarborough and dumb as a rock Mika are not bad people, but their low rated show is dominated by their NBC bosses. Too bad! 2017-07-01T13:20:35.000Z company NBCUniversal ticker CMCSA sentiment -0.4 change -0.15 profit 0.15 Dow hit a new intraday all-time high! I wonder whether or not the Fake News Media will so report? 2017-07-03T21:10:25.000Z company Dow Jones Industrial Average ticker NWSA sentiment -0.3 change -0.04 profit 0.04 Join me live for the commissioning ceremony of the USS Gerald R. Ford!
➡️https://t.co/F78iOR64TW #USA https://t.co/PrVceEmPOB 2017-07-22T14:43:18.000Z company Newport News Shipbuilding ticker HII sentiment 0.1 change -0.02 profit -0.02 So many stories about me in the @washingtonpost are Fake News. They are as bad as ratings challenged @CNN. Lobbyist for Amazon and taxes? 2017-07-25T02:28:44.000Z company Amazon ticker AMZN sentiment -0.3 change -0.07 profit 0.07 Is Fake News Washington Post being used as a lobbyist weapon against Congress to keep Politicians from looking into Amazon no-tax monopoly? 2017-07-25T02:36:17.000Z company Amazon ticker AMZN sentiment -0.8 change -0.07 profit 0.07 Attorney General Jeff Sessions has taken a VERY weak position on Hillary Clinton crimes (where are E-mails & DNC server) & Intel leakers! 2017-07-25T10:12:24.000Z company Intel ticker INTC sentiment -0.3 change 0.04 profit -0.04 Toyota & Mazda to build a new $1.6B plant here in the U.S.A. and create 4K new American jobs. A great investment in American manufacturing! 2017-08-04T10:02

In [24]:
(apply (juxt min max) (map :percent-change trades))

[-0.12182741116751272 0.09148089908426593]

In [7]:
(let[[offset slope] (transduce identity (kixi.stats.core/simple-linear-regression :sentiment :percent-change) trades) 
      s1-line (map (juxt identity #(+ offset (* % slope))) (into [-1] (conj (mapv :sentiment trades) 1)))]
    (def spec
         {:x-axis (viz/linear-axis
                   {:domain      [-1 1]
                    :range       [100 850]
                    :major       0.1
;;                  :minor       0.05
                    :pos         550
                    :label-dist  15
                    :label-style {:text-anchor "end"}})
          :y-axis (viz/linear-axis
                   {:domain      [-1 1]
                    :range       [550 50]
                    :major       0.1
;;                  :minor       0.005
                    :pos         100
                    :label-dist  15
                    :label-style {:text-anchor "end"}})
          :grid   {:attribs {:stroke "#caa"}
                   :minor-x true
                   :minor-y true}
          :data   [{:values  (map (juxt :sentiment :percent-change) trades)
                    :attribs {:fill "#0af" :stroke "none"}
                    :layout  viz/svg-scatter-plot}
                   {:values  s1-line
                    :attribs {:fill "none" :stroke "#0fa" :stroke-width "3px"}
                    :layout  viz/svg-line-plot}]}))
          
        
                

#'trachenberg.notebook.playground/spec

In [8]:
(export-viz spec)

<?xml version="1.0"?>
 <polyline fill="none" points="100.00,312.63 137.50,311.22 175.00,309.81 175.00,309.81 212.50,308.40 212.50,308.40 250.00,306.99 250.00,306.99 250.00,306.99 250.00,306.99 250.00,306.99 287.50,305.58 287.50,305.58 325.00,304.17 325.00,304.17 325.00,304.17 325.00,304.17 325.00,304.17 325.00,304.17 362.50,302.76 362.50,302.76 362.50,302.76 362.50,302.76 362.50,302.76 362.50,302.76 400.00,301.35 400.00,301.35 400.00,301.35 400.00,301.35 437.50,299.94 512.50,297.12 512.50,297.12 512.50,297.12 512.50,297.12 512.50,297.12 512.50,297.12 550.00,295.71 587.50,294.30 587.50,294.30 587.50,294.30 587.50,294.30 587.50,294.30 587.50,294.30 587.50,294.30 587.50,294.30 587.50,294.30 587.50,294.30 587.50,294.30 625.00,292.89 625.00,292.89 625.00,292.89 662.50,291.48 662.50,291.48 700.00,290.07 737.50,288.66 737.50,288.66 850.00,284.43" stroke-width="3px" stroke="#0fa" /> -1.00 -0.90 -0.80 -0.70 -0.60 -0.50 -0.40 -0.30 -0.20 -0.10 -0.00 0.10 0.20 0.30 0.40 0.50 0.60 0.70 0.80 0.90 1.00 -1.00 -0.90 -0.80 -0.70 -0.60 -0.50 -0.40 -0.30 -0.20 -0.10 -0.00 0.10 0.20 0.30 0.40 0.50 0.60 0.70 0.80 0.90 1.00

In [5]:
(let[[offset slope] (transduce identity (kixi.stats.core/simple-linear-regression :sentiment :profit) trades) 
      s1-line (map (juxt identity #(+ offset (* % slope))) (into [-1] (conj (mapv :sentiment trades) 1)))]
    (def spec
         {:x-axis (viz/linear-axis
                   {:domain      [-1 1]
                    :range       [100 850]
                    :major       0.1
;;                  :minor       0.05
                    :pos         550
                    :label-dist  15
                    :label-style {:text-anchor "end"}})
          :y-axis (viz/linear-axis
                   {:domain      [-1 1]
                    :range       [550 50]
                    :major       0.1
;;                  :minor       0.005
                    :pos         100
                    :label-dist  15
                    :label-style {:text-anchor "end"}})
          :grid   {:attribs {:stroke "#caa"}
                   :minor-x true
                   :minor-y true}
          :data   [{:values  (map (juxt :sentiment :profit) trades)
                    :attribs {:fill "#0af" :stroke "none"}
                    :layout  viz/svg-scatter-plot}
                   {:values  s1-line
                    :attribs {:fill "none" :stroke "#0fa" :stroke-width "3px"}
                    :layout  viz/svg-line-plot}]}))
          
        
                

#'trachenberg.notebook.playground/spec

In [6]:
(export-viz spec)

<?xml version="1.0"?>
 <polyline fill="none" points="100.00,288.60 137.50,288.76 175.00,288.93 175.00,288.93 212.50,289.09 212.50,289.09 250.00,289.26 250.00,289.26 250.00,289.26 250.00,289.26 250.00,289.26 287.50,289.43 287.50,289.43 325.00,289.59 325.00,289.59 325.00,289.59 325.00,289.59 325.00,289.59 325.00,289.59 362.50,289.76 362.50,289.76 362.50,289.76 362.50,289.76 362.50,289.76 362.50,289.76 400.00,289.92 400.00,289.92 400.00,289.92 400.00,289.92 437.50,290.09 512.50,290.42 512.50,290.42 512.50,290.42 512.50,290.42 512.50,290.42 512.50,290.42 550.00,290.59 587.50,290.76 587.50,290.76 587.50,290.76 587.50,290.76 587.50,290.76 587.50,290.76 587.50,290.76 587.50,290.76 587.50,290.76 587.50,290.76 587.50,290.76 625.00,290.92 625.00,290.92 625.00,290.92 662.50,291.09 662.50,291.09 700.00,291.25 737.50,291.42 737.50,291.42 850.00,291.92" stroke-width="3px" stroke="#0fa" /> -1.00 -0.90 -0.80 -0.70 -0.60 -0.50 -0.40 -0.30 -0.20 -0.10 -0.00 0.10 0.20 0.30 0.40 0.50 0.60 0.70 0.80 0.90 1.00 -1.00 -0.90 -0.80 -0.70 -0.60 -0.50 -0.40 -0.30 -0.20 -0.10 -0.00 0.10 0.20 0.30 0.40 0.50 0.60 0.70 0.80 0.90 1.00